In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
# cifar-10官方提供的数据集是用numpy array存储的
# 下面这个transform会把numpy array变成torch tensor，然后把rgb值归一到[0, 1]这个区间
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# 在构建数据集的时候指定transform，就会应用我们定义好的transform
# root是存储数据的文件夹，download=True指定如果数据不存在先下载数据
cifar_train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
cifar_test = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform)


Files already downloaded and verified


In [4]:
# load完了之后我们看看这两个数据集的信息
print(cifar_train)
print(cifar_test)

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )
Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )


In [10]:
# 数据其实是用numpy array存储的，label是个list

# 旧版
# print(cifar_train.train_data.shape)
# print(type(cifar_train.train_labels), len(cifar_train.train_labels))
# print(cifar_test.test_data.shape)
# print(type(cifar_test.test_labels), len(cifar_test.test_labels))

# 新版
print(cifar_train.data.shape)
print(type(cifar_train.targets), len(cifar_train.targets))
print(cifar_test.data.shape)
print(type(cifar_test.targets), len(cifar_test.targets))

(50000, 32, 32, 3)
<class 'list'> 50000
(10000, 32, 32, 3)
<class 'list'> 10000


In [12]:
# 在训练的时候我们可以自己写代码手动遍历数据集，指定batch和遍历方法，不过PyTorch提供了一个DataLoader类来方便我们完成这些操作。
trainloader = torch.utils.data.DataLoader(cifar_train, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(cifar_test, batch_size=32, shuffle=True)

现在我们来定义卷积神经网络，为了简单起见，我们使用经典的LeNet，它包含两个卷积层和三个全连接层，网络结构如图
![](https://pic4.zhimg.com/80/v2-153a890f1412bea4542d568d2f071fc7_720w.webp)

In [ ]:
# 在PyTorch中定义神经网络非常简单，第一步先是继承nn.module这个类，然后定义如下两个函数，一般的网络这样操作就足够了
class LeNet(nn.Module):
    # 一般在__init__中定义网络需要的操作算子，比如卷积、全连接算子等等
    def __init__(self):
        super(LeNet, self).__init__()
        # Conv2d的第一个参数是输入的channel数量，第二个是输出的channel数量，第三个是kernel size
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 由于上一层有16个channel输出，每个feature map大小为5*5，所以全连接层的输入是16*5*5
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        # 最终有10类，所以最后一个全连接层输出数量是10
        self.fc3 = nn.Linear(84, 10)
        self.pool = nn.MaxPool2d(2, 2)
    # forward这个函数定义了前向传播的运算，只需要像写普通的python算数运算那样就可以了
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        # 下面这步把二维特征图变为一维，这样全连接层才能处理
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x